In [1]:
from transliterate import translit, get_available_language_codes
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

import random
import re

# 1. Подмосковные города с населением от 40 тыс - готовим список (полуручник)

In [ ]:
s="""Балашиха - 197,7

Егорьевск - 66,9

Зарайск - 23,8

Химки - 188,5

Ступино - 65,9

Волоколамск - 23,6

Подольск - 183,1

Дмитров - 63,3

Лосино-Петровский - 22,4

Королев - 176,0

Дубна - 62,7

Котельники - 21,8

Мытищи - 164,6

Павловский Посад - 61,8

Старая Купавна - 21,2

Люберцы - 158,2

Ивантеевка - 57,2

Электрогорск - 21,1

Коломна - 148,4

Солнечногорск - 57,1

Рошаль - 21,0

Электросталь - 146,4

Климовск - 56,1

Черноголовка - 20,5

Железнодорожный - 128,5

Видное - 55,1

Пущино - 20,3

Серпухов - 123,1

Фрязино - 53,2

Хотьково - 19,8

Орехово-Зуево - 121,1

Лыткарино - 52,1

Электроугли - 19,6

Ногинск - 115,9

Дзержинский - 45,0

Бронницы - 19,0

Одинцово - 109,6

Кашира - 39,1

Апрелевка - 18,7

Щелково - 108,1

Троицк - 37,6

Куровское - 18,5

Сергиев Посад - 105,8

Протвино - 37,2

Голицыно - 17,5

Жуковский - 104,8

Краснознаменск - 33,8

Московский - 15,6

Красногорск - 104,2

Юбилейный - 32,5

Пересвет - 14,5

Пушкино - 98,9

Щербинка - 32,3

Краснозаводск - 13,7

Воскресенск - 90,8

Истра - 32,0

Руза - 13,1

Домодедово - 89,8

Луховицы - 31,8

Яхрома - 13,1

Догопрудный - 84,4

Ликино-Дулево - 31,0

Звенигород - 12,9

Раменское - 83,7

Шатура - 30,8

Талдом - 12,3

Реутов - 83,6

Можайск - 30,2

Дрезна - 11,5

Клин - 80,3

Дедовск - 28,2

Высоковск - 10,7

Чехов - 73,4

Красноармейск - 26,3

Ожерелье - 10,6

Наро-Фоминск - 71,1

Кубинка - 26,2

Верея - 4,7

Лобня - 71,0

Озеры - 25,7
"""


In [ ]:
s=s.replace(',','.')

L2=[]
for i,v in enumerate(s.split('\n')):
    if i%2==0:
        L2.append(v)
L3=[]
for _ in L2:
    L3.append(_.split('-'))

In [208]:
for l in L3:
    if len(l)>2:
        print(l)

In [ ]:
L3.index(['Орехово', 'Зуево ', ' 121.1']), L3.index(['Ликино', 'Дулево ', ' 31.0'])

In [ ]:
del(L3[8])
del(L3[-5])
del L3[60], L3[29]

In [ ]:
L3.append(['Наро-Фоминск', '71.1']), L3.append(['Орехово-Зуево ', ' 121.1'])

In [ ]:
df_mo_cities=pd.DataFrame(L3, columns=['city', 'population'])

In [ ]:
df_mo_cities['city']=df_mo_cities['city'].apply(lambda x: x.rstrip())
df_mo_cities['population']=df_mo_cities['population'].apply(lambda x: x.lstrip()).astype('float')

df_mo_cities['city_eng']=df_mo_cities.city.apply(lambda x: translit(x, 'ru', reversed=True)
                                                ).apply(lambda x: x.replace("'", ""))

In [ ]:
cities_eng=df_mo_cities[df_mo_cities.population>=40].sort_values(by='population', ascending=False)['city_eng'].unique()
cities_eng

In [ ]:
d={}
for c in cities_eng:
    d[c]=str(requests.get("https://mainfin.ru/banki/poisk/bankomaty"+f"/{c}"))

L=[]
for i in d.items():
    L.append(i)

df_resp=pd.DataFrame(L, columns=['city_eng','response'])

In [167]:
LL=[
'https://mainfin.ru/banki/poisk/bankomaty/sergiev-posad',
'https://mainfin.ru/banki/poisk/bankomaty/mitischi',
'https://mainfin.ru/banki/poisk/bankomaty/luberci',
'https://mainfin.ru/banki/poisk/bankomaty/zheleznodorozhniy',
'https://mainfin.ru/banki/poisk/bankomaty/odincovo',
'https://mainfin.ru/banki/poisk/bankomaty/zhukovskiy',
'https://mainfin.ru/banki/poisk/bankomaty/dolgoprudniy',
'https://mainfin.ru/banki/poisk/bankomaty/lobnya',
'https://mainfin.ru/banki/poisk/bankomaty/moskovskaja-oblast-dubna',
'https://mainfin.ru/banki/poisk/bankomaty/pavlovskiy-posad',
'https://mainfin.ru/banki/poisk/bankomaty/moskovskaja-oblast-ivanteevka',
'https://mainfin.ru/banki/poisk/bankomaty/fryazino',
'https://mainfin.ru/banki/poisk/bankomaty/litkarino',
'https://mainfin.ru/banki/poisk/bankomaty/dzerzhinskiy']

In [ ]:
for c in df_resp[df_resp.response=='<Response [200]>'].city_eng.unique():
    LL.append("https://mainfin.ru/banki/poisk/bankomaty"+f"/{c}")

In [33]:
#для повторных прогрузок, чтобы не повторять сборку выше, берем список из аутпута ниже
LL=['https://mainfin.ru/banki/poisk/bankomaty/sergiev-posad',
 'https://mainfin.ru/banki/poisk/bankomaty/mitischi',
 'https://mainfin.ru/banki/poisk/bankomaty/luberci',
 'https://mainfin.ru/banki/poisk/bankomaty/zheleznodorozhniy',
 'https://mainfin.ru/banki/poisk/bankomaty/odincovo',
 'https://mainfin.ru/banki/poisk/bankomaty/zhukovskiy',
 'https://mainfin.ru/banki/poisk/bankomaty/dolgoprudniy',
 'https://mainfin.ru/banki/poisk/bankomaty/lobnya',
 'https://mainfin.ru/banki/poisk/bankomaty/moskovskaja-oblast-dubna',
 'https://mainfin.ru/banki/poisk/bankomaty/pavlovskiy-posad',
 'https://mainfin.ru/banki/poisk/bankomaty/moskovskaja-oblast-ivanteevka',
 'https://mainfin.ru/banki/poisk/bankomaty/fryazino',
 'https://mainfin.ru/banki/poisk/bankomaty/litkarino',
 'https://mainfin.ru/banki/poisk/bankomaty/dzerzhinskiy',
 'https://mainfin.ru/banki/poisk/bankomaty/Balashiha',
 'https://mainfin.ru/banki/poisk/bankomaty/Himki',
 'https://mainfin.ru/banki/poisk/bankomaty/Podolsk',
 'https://mainfin.ru/banki/poisk/bankomaty/Korolev',
 'https://mainfin.ru/banki/poisk/bankomaty/Kolomna',
 'https://mainfin.ru/banki/poisk/bankomaty/Elektrostal',
 'https://mainfin.ru/banki/poisk/bankomaty/Serpuhov',
 'https://mainfin.ru/banki/poisk/bankomaty/Orehovo-Zuevo',
 'https://mainfin.ru/banki/poisk/bankomaty/Noginsk',
 'https://mainfin.ru/banki/poisk/bankomaty/Schelkovo',
 'https://mainfin.ru/banki/poisk/bankomaty/Krasnogorsk',
 'https://mainfin.ru/banki/poisk/bankomaty/Voskresensk',
 'https://mainfin.ru/banki/poisk/bankomaty/Domodedovo',
 'https://mainfin.ru/banki/poisk/bankomaty/Ramenskoe',
 'https://mainfin.ru/banki/poisk/bankomaty/Reutov',
 'https://mainfin.ru/banki/poisk/bankomaty/Klin',
 'https://mainfin.ru/banki/poisk/bankomaty/Chehov',
 'https://mainfin.ru/banki/poisk/bankomaty/Naro-Fominsk',
 'https://mainfin.ru/banki/poisk/bankomaty/Egorevsk',
 'https://mainfin.ru/banki/poisk/bankomaty/Stupino',
 'https://mainfin.ru/banki/poisk/bankomaty/Dmitrov',
 'https://mainfin.ru/banki/poisk/bankomaty/Solnechnogorsk',
 'https://mainfin.ru/banki/poisk/bankomaty/Klimovsk',
 'https://mainfin.ru/banki/poisk/bankomaty/Vidnoe']

# 2. По готовому списку url городов собираем инфо о банкоматах со всех страниц  для каждого города

In [23]:
adr=[]
bank_name=[]
schedule=[]
for u in LL:
    response=requests.get(u)
    soupB = BeautifulSoup(response.text, 'html', multi_valued_attributes=None)
    
    #Смотрим сколько банкоматов в городе (нужно для определения кол-ва страниц на город (20 банкоматов на 1 странице)
    s_count=soupB.findAll('label', {'class':"mapobject-count"})[0].get_text()
    s_count=int(''.join(re.findall(r'\d', s_count)))
    n_pages=s_count//20

    for p in range(1, n_pages):
        url=u+"?page={}".format(p)
        page=requests.get(url)
        soup = BeautifulSoup(page.text, 'html', multi_valued_attributes=None)
    
        for _ in soup.find_all('a', {'class':'bank small'}):
            bank_name.append(_.get_text())
    
        for _ in soup.find_all('a', {'class':'address pm'}):
            adr.append(_.get_text().replace('\xa0', ''))
    
        for _ in soup.find_all('td', {'class':'col-sm-3'}):
            schedule.append(_.get_text())
    
        sleep(random.random())
        
    sleep(random.random())

In [24]:
df=pd.DataFrame(zip(adr, bank_name, schedule), columns=['address','bank_name','schedule'])

In [25]:
df.head()

,address,bank_name,schedule
0,"г. Сергиев Посад, ул.Кооперативная, д.2",Авангард,Круглосуточно
1,"г. Сергиев Посад, ул.Вознесенская, д.32а",Банк Уралсиб,Круглосуточно
2,"г. Сергиев Посад, ул.1-й Ударной Армии, д.12",Банк Уралсиб,Круглосуточно
3,"г. Сергиев Посад, просп. Красной Армии, д.4",Банк Уралсиб,Круглосуточно
4,"г. Сергиев Посад, ш.Новоугличское, д.46",Банк «Открытие»,Круглосуточно


In [17]:
u='https://mainfin.ru/banki/poisk/bankomaty/moskva'

adr=[]
bank_name=[]
schedule=[]

response=requests.get(u)
soupB = BeautifulSoup(response.text, 'html', multi_valued_attributes=None)

#Смотрим сколько банкоматов в городе (нужно для определения кол-ва страниц на город (20 банкоматов на 1 странице)
s_count=soupB.findAll('label', {'class':"mapobject-count"})[0].get_text()
s_count=int(''.join(re.findall(r'\d', s_count)))
n_pages=s_count//20

for p in range(1, n_pages):
    url=u+"?page={}".format(p)
    page=requests.get(url)
    soup = BeautifulSoup(page.text, 'html', multi_valued_attributes=None)

    for _ in soup.find_all('a', {'class':'bank small'}):
        bank_name.append(_.get_text())

    for _ in soup.find_all('a', {'class':'address pm'}):
        adr.append(_.get_text().replace('\xa0', ''))

    for _ in soup.find_all('td', {'class':'col-sm-3'}):
        schedule.append(_.get_text())

    sleep(random.random())


In [18]:
df_msk=pd.DataFrame(zip(adr, bank_name, schedule), columns=['address','bank_name','schedule'])

In [20]:
df_msk.head()

,address,bank_name,schedule
0,"г. Москва, просп. Кутузовский, д.24",Абсолют Банк,Круглосуточно
1,"г. Москва, ул.Садовая-Черногрязская, д.3 Б, ст...",Абсолют Банк,Круглосуточно
2,"г. Москва, просп. Ленинградский, д.33, корп. 3",Абсолют Банк,Круглосуточно
3,"г. Москва, ул.Люблинская, д.165",Абсолют Банк,Круглосуточно
4,"г. Москва, ул.Южнобутовская, д.61",Абсолют Банк,Доступен в режиме работы места установки


In [30]:
df_MMO=pd.concat([df, df_msk], ignore_index=True)

In [32]:
df_MMO.to_csv('atms_MMO.csv', index=False)